# Optimization Model 

In [2]:
# Import dependencies
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Load the dataset
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


#### Preprocessing

In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME' 'STATUS', 'SPECIAL CONSIDERATIONS'
application_df = application_df.drop(columns=['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [5]:
# Determine the number of unique values in each column
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [6]:
# Look at application_type value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [ ]:
# Application Types
# Combine rare categorical variables for columns with more than 10 unique values
application_types_to_replace = application_counts[application_counts < 500].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, 'Other')

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [9]:
# Classification Types
# Combine rare categorical variables for columns with more than 10 unique values
classification_types_to_replace = classification_counts[classification_counts < 1000].index

# Replace in dataframe
for cls in classification_types_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, 'Other')
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,108590,1,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,5000,0,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
3,6692,1,False,False,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
4,142590,1,False,False,False,True,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False


In [11]:
# Split the preprocessed data into features (X) and target (y)
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the training data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### 1st Optimization Attempt: 

* Accuracy: 73% 
* Loss: 56%

In [15]:
# Define the neural network model 
input_features = X_train_scaled.shape[1]

optimized_model = tf.keras.models.Sequential()

# Add the first hidden layer with more neurons and L2 regularization
optimized_model.add(Dense(units=128, activation='relu', input_dim=input_features, kernel_regularizer=tf.keras.regularizers.l2(0.01)))
optimized_model.add(Dropout(0.3))  # Dropout to prevent overfitting

# Add the second hidden layer with more neurons and L2 regularization
optimized_model.add(Dense(units=64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
optimized_model.add(Dropout(0.3))

# Add a third hidden layer
optimized_model.add(Dense(units=32, activation='relu'))

# Add the output layer
optimized_model.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the optimized model
optimized_model.summary()

/opt/anaconda3/envs/env1/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,617 (61.00 KB)

 Trainable params: 15,617 (61.00 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Compile the optimized model
optimized_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Import new dependency (to help with callback)
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Create a callback that saves the model's weights
checkpoint = ModelCheckpoint(filepath="optimized_weights.{epoch:02d}.weights.h5", save_weights_only=True, save_freq=5*len(X_train_scaled))
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

In [20]:
# Train the optimized model
optimized_history = optimized_model.fit(
    X_train_scaled, y_train, 
    epochs=150,  # Increased epochs for better learning
    batch_size=32, 
    callbacks=[checkpoint, reduce_lr], 
    validation_split=0.2
)

Epoch 1/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 593us/step - accuracy: 0.6857 - loss: 1.2102 - val_accuracy: 0.7318 - val_loss: 0.5947 - learning_rate: 0.0010
Epoch 2/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step - accuracy: 0.7212 - loss: 0.5989 - val_accuracy: 0.7367 - val_loss: 0.5727 - learning_rate: 0.0010
Epoch 3/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 499us/step - accuracy: 0.7240 - loss: 0.5893 - val_accuracy: 0.7345 - val_loss: 0.5752 - learning_rate: 0.0010
Epoch 4/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step - accuracy: 0.7230 - loss: 0.5868 - val_accuracy: 0.7360 - val_loss: 0.5721 - learning_rate: 0.0010
Epoch 5/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step - accuracy: 0.7232 - loss: 0.5858 - val_accuracy: 0.7387 - val_loss: 0.5700 - learning_rate: 0.0010
Epoch 6/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step - accuracy: 0.7246 - loss: 0.5844 - val_accuracy: 0.7362 - val_loss: 0.5727 - learning_rate: 0.0010
Epoch 7/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 497us/step - accuracy: 0.7

In [21]:
# Evaluate the optimized model using test data
optimized_loss, optimized_accuracy = optimized_model.evaluate(X_test_scaled, y_test)
print(f"Optimized Test Loss: {optimized_loss}")
print(f"Optimized Test Accuracy: {optimized_accuracy}")

215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - accuracy: 0.7302 - loss: 0.5592
Optimized Test Loss: 0.56125408411026
Optimized Test Accuracy: 0.7300291657447815


#### 2nd Optimization Attempt
* Accuracy: 72% (decrease since 1st attempt)
* Loss: 56%

In [26]:
# Import LeakyReLU (to better optimize the model)
from tensorflow.keras.layers import LeakyReLU

# Define the Model 
input_features = X_train_scaled.shape[1]

optimized_2 = tf.keras.models.Sequential()

# Add the first hidden layer with more neurons and LeakyReLU activation
optimized_2.add(Dense(units=256, input_dim=input_features, kernel_regularizer=tf.keras.regularizers.l2(0.01)))
optimized_2.add(LeakyReLU(alpha=0.1))  # LeakyReLU for better gradient flow
optimized_2.add(Dropout(0.4))  # Increased dropout to prevent overfitting

# Add the second hidden layer with more neurons and LeakyReLU activation
optimized_2.add(Dense(units=128, kernel_regularizer=tf.keras.regularizers.l2(0.01)))
optimized_2.add(LeakyReLU(alpha=0.1))
optimized_2.add(Dropout(0.3))

# Add a third hidden layer
optimized_2.add(Dense(units=64, activation='relu'))

# Add a fourth hidden layer
optimized_2.add(Dense(units=32, activation='relu'))

# Add the output layer
optimized_2.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the optimized model
optimized_2.summary()

/opt/anaconda3/envs/env1/lib/python3.11/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 256)            │        10,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,761 (210.00 KB)

 Trainable params: 53,761 (210.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the optimized model with a lower learning rate
optimized_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Create callbacks
checkpoint = ModelCheckpoint(filepath="optimized_2_weights.{epoch:02d}.weights.h5", save_weights_only=True, save_freq=5*len(X_train_scaled))
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Train the 2nd optimized mode
optimized_history = optimized_2.fit(
    X_train_scaled, y_train, 
    epochs=200,  # Increased epochs for better learning
    batch_size=32, 
    callbacks=[checkpoint, reduce_lr], 
    validation_split=0.2
)



Epoch 1/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 829us/step - accuracy: 0.6787 - loss: 1.6701 - val_accuracy: 0.7351 - val_loss: 0.6229 - learning_rate: 5.0000e-04
Epoch 2/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 722us/step - accuracy: 0.7213 - loss: 0.6127 - val_accuracy: 0.7354 - val_loss: 0.5787 - learning_rate: 5.0000e-04
Epoch 3/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 718us/step - accuracy: 0.7242 - loss: 0.5904 - val_accuracy: 0.7411 - val_loss: 0.5685 - learning_rate: 5.0000e-04
Epoch 4/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 723us/step - accuracy: 0.7274 - loss: 0.5801 - val_accuracy: 0.7301 - val_loss: 0.5727 - learning_rate: 5.0000e-04
Epoch 5/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 722us/step - accuracy: 0.7249 - loss: 0.5783 - val_accuracy: 0.7371 - val_loss: 0.5679 - learning_rate: 5.0000e-04
Epoch 6/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 757us/step - accuracy: 0.7277 - loss: 0.5787 - val_accuracy: 0.7356 - val_loss: 0.5748 - learning_rate: 5.0000e-04
Epoch 7/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 71

In [34]:
# Evaluate the optimized model using the test data
optimized_loss, optimized_accuracy = optimized_2.evaluate(X_test_scaled, y_test)
print(f"Optimized Test Loss: {optimized_loss}")
print(f"Optimized Test Accuracy: {optimized_accuracy}")

215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step - accuracy: 0.7274 - loss: 0.5642
Optimized Test Loss: 0.565542459487915
Optimized Test Accuracy: 0.7244898080825806


#### 3rd Optimization Attempt
Droping another column and using more paramters to see if this achieves better accuracy. 

* Accuracy: 72%
* Loss: 55%

In [ ]:
# Drop the ASK_AMT column (potentially may have outliers)
application_df = application_df.drop(columns=['ASK_AMT'])

# Check the number of unique values in each column
application_df.nunique()

IS_SUCCESSFUL                   2
APPLICATION_TYPE_Other          2
APPLICATION_TYPE_T10            2
APPLICATION_TYPE_T19            2
APPLICATION_TYPE_T3             2
APPLICATION_TYPE_T4             2
APPLICATION_TYPE_T5             2
APPLICATION_TYPE_T6             2
APPLICATION_TYPE_T7             2
APPLICATION_TYPE_T8             2
AFFILIATION_CompanySponsored    2
AFFILIATION_Family/Parent       2
AFFILIATION_Independent         2
AFFILIATION_National            2
AFFILIATION_Other               2
AFFILIATION_Regional            2
CLASSIFICATION_C1000            2
CLASSIFICATION_C1200            2
CLASSIFICATION_C2000            2
CLASSIFICATION_C2100            2
CLASSIFICATION_C3000            2
CLASSIFICATION_Other            2
USE_CASE_CommunityServ          2
USE_CASE_Heathcare              2
USE_CASE_Other                  2
USE_CASE_Preservation           2
USE_CASE_ProductDev             2
ORGANIZATION_Association        2
ORGANIZATION_Co-operative       2
ORGANIZATION_C

In [38]:
# Use a differnt Model Architecture (3rd Attempt)
# Import BatchNormalization (to better optimize the model)
from tensorflow.keras.layers import BatchNormalization

# Define the Model 
input_features = X_train_scaled.shape[1]

optimized_3 = tf.keras.models.Sequential()

# Add the first hidden layer
optimized_3.add(Dense(units=256, activation='relu', input_dim=input_features))
optimized_3.add(BatchNormalization())  # Normalize the activations
optimized_3.add(Dropout(0.3))  # Dropout to prevent overfitting

# Add the second hidden layer
optimized_3.add(Dense(units=128, activation='relu'))
optimized_3.add(BatchNormalization())
optimized_3.add(Dropout(0.3))

# Add a third hidden layer
optimized_3.add(Dense(units=64, activation='relu'))
optimized_3.add(BatchNormalization())
optimized_3.add(Dropout(0.2))

# Add a fourth hidden layer
optimized_3.add(Dense(units=32, activation='relu'))
optimized_3.add(BatchNormalization())
optimized_3.add(Dropout(0.2))

# Add the output layer
optimized_3.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the optimized model
optimized_3.summary()

/opt/anaconda3/envs/env1/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 256)            │        10,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 55,681 (217.50 KB)

 Trainable params: 54,721 (213.75 KB)

 Non-trainable params: 960 (3.75 KB)

In [40]:
# Compile the model
optimized_3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), 
                    loss='binary_crossentropy', 
                    metrics=['accuracy'])

# Train the model
history_3 = optimized_3.fit(
    X_train_scaled, y_train, 
    epochs=150,  # Train for more epochs
    batch_size=32, 
    validation_split=0.2, 
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001),
        tf.keras.callbacks.ModelCheckpoint(filepath="optimized_3_weights.{epoch:02d}.weights.h5", save_weights_only=True, save_freq=5*len(X_train_scaled))
    ]
)

Epoch 1/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 998us/step - accuracy: 0.6356 - loss: 0.7376 - val_accuracy: 0.7316 - val_loss: 0.5594 - learning_rate: 5.0000e-04
Epoch 2/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 840us/step - accuracy: 0.6885 - loss: 0.6145 - val_accuracy: 0.7374 - val_loss: 0.5555 - learning_rate: 5.0000e-04
Epoch 3/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 830us/step - accuracy: 0.7096 - loss: 0.5847 - val_accuracy: 0.7378 - val_loss: 0.5512 - learning_rate: 5.0000e-04
Epoch 4/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 846us/step - accuracy: 0.7175 - loss: 0.5773 - val_accuracy: 0.7382 - val_loss: 0.5514 - learning_rate: 5.0000e-04
Epoch 5/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 843us/step - accuracy: 0.7157 - loss: 0.5775 - val_accuracy: 0.7372 - val_loss: 0.5492 - learning_rate: 5.0000e-04
Epoch 6/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 882us/step - accuracy: 0.7196 - loss: 0.5708 - val_accuracy: 0.7354 - val_loss: 0.5495 - learning_rate: 5.0000e-04
Epoch 7/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 82

In [41]:
# Evaluate the model
loss_3, accuracy_3 = optimized_3.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {loss_3}")
print(f"Test Accuracy: {accuracy_3}")

215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 398us/step - accuracy: 0.7250 - loss: 0.5583
Test Loss: 0.5587342977523804
Test Accuracy: 0.7258017659187317


#### 4th Optimization Attempt
Trying out Hyperparameter Tuning to see if that achieves better accuracy. 
* Accuracy: 72%
* Loss: 55%

In [44]:
# Importing Keras Tuner
from kerastuner.tuners import RandomSearch
from tensorflow.keras.optimizers import Adam

In [46]:
# Define a function to build the model
def build_optimized_4(hp):
    optimized_4 = Sequential()
    
    # Input layer and first hidden layer
    optimized_4.add(Dense(units=hp.Int('units_layer_1', min_value=64, max_value=256, step=32), 
                          activation='relu', input_dim=X_train_scaled.shape[1]))
    optimized_4.add(BatchNormalization())
    optimized_4.add(Dropout(hp.Float('dropout_layer_1', min_value=0.2, max_value=0.5, step=0.1)))
    
    # Second hidden layer
    optimized_4.add(Dense(units=hp.Int('units_layer_2', min_value=32, max_value=128, step=32), activation='relu'))
    optimized_4.add(BatchNormalization())
    optimized_4.add(Dropout(hp.Float('dropout_layer_2', min_value=0.2, max_value=0.5, step=0.1)))

    # Third hidden layer (optional, based on hyperparameter tuning)
    if hp.Boolean('add_layer_3'):
        optimized_4.add(Dense(units=hp.Int('units_layer_3', min_value=16, max_value=64, step=16), activation='relu'))
        optimized_4.add(BatchNormalization())
        optimized_4.add(Dropout(hp.Float('dropout_layer_3', min_value=0.2, max_value=0.5, step=0.1)))
    
    # Output layer
    optimized_4.add(Dense(units=1, activation='sigmoid'))
    
     # Compile the model
    optimized_4.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.0005, 0.0001])),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])
    
    return optimized_4

In [48]:
# Initialize the Keras Tuner
tuner = RandomSearch(
    build_optimized_4,
    objective='val_accuracy',
    max_trials=10,  # Number of different hyperparameter combinations to try
    executions_per_trial=2,  # Number of models to train per trial for stability
    directory='my_dir',
    project_name='alphabet_soup_tuning'
)

# Perform the search
tuner.search(X_train_scaled, y_train, epochs=50, validation_split=0.2, verbose=1)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 10 Complete [00h 00m 50s]
val_accuracy: 0.7408892214298248

Best val_accuracy So Far: 0.7417092025279999
Total elapsed time: 00h 07m 45s


In [49]:
# Print the best hyperparameters
print(f"""
The optimal number of units in the first hidden layer is {best_hps.get('units_layer_1')}.
The optimal number of units in the second hidden layer is {best_hps.get('units_layer_2')}.
The optimal dropout rate for the first hidden layer is {best_hps.get('dropout_layer_1')}.
The optimal dropout rate for the second hidden layer is {best_hps.get('dropout_layer_2')}.
The optimal learning rate is {best_hps.get('learning_rate')}.
""")


The optimal number of units in the first hidden layer is 256.
The optimal number of units in the second hidden layer is 128.
The optimal dropout rate for the first hidden layer is 0.30000000000000004.
The optimal dropout rate for the second hidden layer is 0.2.
The optimal learning rate is 0.001.



In [50]:
# Build the model with the best hyperparameters
optimized_4 = tuner.hypermodel.build(best_hps)

# Train the best model
history_4 = optimized_4.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 881us/step - accuracy: 0.6683 - loss: 0.6586 - val_accuracy: 0.7352 - val_loss: 0.5556
Epoch 2/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 767us/step - accuracy: 0.7159 - loss: 0.5766 - val_accuracy: 0.7362 - val_loss: 0.5486
Epoch 3/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 781us/step - accuracy: 0.7211 - loss: 0.5740 - val_accuracy: 0.7367 - val_loss: 0.5524
Epoch 4/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 770us/step - accuracy: 0.7319 - loss: 0.5609 - val_accuracy: 0.7365 - val_loss: 0.5506
Epoch 5/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 844us/step - accuracy: 0.7285 - loss: 0.5610 - val_accuracy: 0.7389 - val_loss: 0.5515
Epoch 6/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 760us/step - accuracy: 0.7233 - loss: 0.5649 - val_accuracy: 0.7347 - val_loss: 0.5480
Epoch 7/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 778us/step - accuracy: 0.7232 - loss: 0.5581 - val_accuracy: 0.7392 - val_loss: 0.5489
Epoch 8/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 769us/step - accuracy: 0.7348 - loss: 0

In [51]:
# Evaluate the best model
loss_4, accuracy_4 = optimized_4.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {loss_4}")
print(f"Test Accuracy: {accuracy_4}")


215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 379us/step - accuracy: 0.7238 - loss: 0.5584
Test Loss: 0.5596018433570862
Test Accuracy: 0.7240524888038635


In [52]:
# Save the best model
optimized_4.save("AlphabetSoupCharity_Optimized_4.h5")